<a href="https://colab.research.google.com/github/zakihir0/first-commit/blob/main/Houseprize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/zakihir0/houseprice.git

Cloning into 'houseprice'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 9 (delta 2), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (9/9), done.


In [4]:
import pandas as pd
import matplotlib.figure as figure
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import WhiteKernel, RBF, ConstantKernel, Matern, DotProduct
from sklearn.model_selection import train_test_split, GridSearchCV

In [9]:
test = pd.read_csv("https://raw.githubusercontent.com/zakihir0/houseprice/main/test.csv", index_col=0)
train = pd.read_csv("https://raw.githubusercontent.com/zakihir0/houseprice/main/train.csv", index_col=0)

In [10]:
train

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500


In [11]:
train.describe()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,46.549315,...,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,161.319273,...,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,0.000000,...,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,0.000000,...,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,1474.000000,...,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


In [ ]:
def processing(data):
  data

In [3]:
class GPRegressor:
    def __init__(self, X, y, testsize=200, fold_number=5):
        self.X = X
        self.y = y
        self.testsize = testsize
        self.fold_number = fold_number
        
    def validation(self):
        X_train, X_test, y_train, y_test = train_test_split(self.X, self.y, test_size=self.testsize, random_state=0)
        X_train = (X_train - X_train.mean(axis=0)) / X_train.std(axis=0, ddof=1)
        y_train = (y_train - y_train.mean()) / y_train.std(ddof=1)
        X_test = (X_test - X_train.mean(axis=0)) / X_train.std(axis=0, ddof=1)
        
        return X_train, X_test, y_train, y_test
    
    def GPR(self):
        X_train, X_test, y_train, y_test = self.validation()
        
        kernels = [ConstantKernel() * DotProduct() + WhiteKernel(),
        ConstantKernel() * RBF() + WhiteKernel(),
        ConstantKernel() * RBF() + WhiteKernel() + ConstantKernel() * DotProduct(),
        ConstantKernel() * RBF(np.ones(X_train.shape[1])) + WhiteKernel(),
        ConstantKernel() * RBF(np.ones(X_train.shape[1])) + WhiteKernel() + ConstantKernel() * DotProduct(),
        ConstantKernel() * Matern(nu=1.5) + WhiteKernel(),
        ConstantKernel() * Matern(nu=1.5) + WhiteKernel() + ConstantKernel() * DotProduct(),
        ConstantKernel() * Matern(nu=0.5) + WhiteKernel(),
        ConstantKernel() * Matern(nu=0.5) + WhiteKernel() + ConstantKernel() * DotProduct(),
        ConstantKernel() * Matern(nu=2.5) + WhiteKernel(),
        ConstantKernel() * Matern(nu=2.5) + WhiteKernel() + ConstantKernel() * DotProduct()]
        
        cv_model = GridSearchCV(GaussianProcessRegressor(alpha=0), {'kernel': kernels}, cv=self.fold_number)
        cv_model.fit(X_train, y_train)
        optimal_kernel = cv_model.best_params_['kernel']
        model = GaussianProcessRegressor(optimal_kernel, alpha=0)
        model.fit(X_train, y_train)
        
        return model
    
    def calcurated(self):
        X_train, X_test, y_train, y_test = self.validation()
        model = self.GPR()
        calculated_y_train = model.predict(X_train) * y_train.std(ddof=1) + y_train.mean()
        
        # r2, RMSE, MAE
        print('r2: {0}'.format(float(1 - sum((y_train - calculated_y_train) ** 2) / sum((y_train - y_train.mean()) ** 2))))
        print('RMSE: {0}'.format(float((sum((y_train - calculated_y_train) ** 2) / len(y_train)) ** 0.5)))
        print('MAE: {0}'.format(float(sum(abs(y_train - calculated_y_train)) / len(y_train))))
        
        return calculated_y_train
        
    def calcplot(self):
        X_train, X_test, y_train, y_test = self.validation()
        calculated_y_train = self.calcurated()
        
        # yy-plot
        plt.rcParams['font.size'] = 18
        plt.figure(figsize=figure.figaspect(1))
        plt.scatter(y_train, calculated_y_train, c='blue')
        y_max = np.max(np.array([np.array(y_train), calculated_y_train]))
        y_min = np.min(np.array([np.array(y_train), calculated_y_train]))
        plt.plot([y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)],
                 [y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)], 'k-')
        plt.ylim(y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min))
        plt.xlim(y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min))
        plt.xlabel('Actual Y')
        plt.ylabel('Calculated Y')
        plt.show()
        
    def prediction(self):
        X_train, X_test, y_train, y_test = self.validation()
        model = self.GPR()
        
        # prediction
        predicted_y_test, predicted_y_test_std = model.predict(X_test, return_std=True)
        predicted_y_test = predicted_y_test * y_train.std(ddof=1) + y_train.mean()
        predicted_y_test_std = predicted_y_test_std * y_train.std(ddof=1)
        
        # r2p, RMSEp, MAEp
        print('r2p: {0}'.format(float(1 - sum((y_test - predicted_y_test) ** 2) / sum((y_test - y_test.mean()) ** 2))))
        print('RMSEp: {0}'.format(float((sum((y_test - predicted_y_test) ** 2) / len(y_test)) ** 0.5)))
        print('MAEp: {0}'.format(float(sum(abs(y_test - predicted_y_test)) / len(y_test))))
        print('')
        return  predicted_y_test, predicted_y_test_std
        
    def pred_yyplot(self):
        predicted_y_test, predicted_y_test_std = self.prediction()
        X_train, X_test, y_train, y_test = self.validation()
        
        # yy-plot
        plt.figure(figsize=figure.figaspect(1))
        plt.scatter(y_test, predicted_y_test, c='blue')
        y_max = np.max(np.array([np.array(y_test), predicted_y_test]))
        y_min = np.min(np.array([np.array(y_test), predicted_y_test]))
        plt.plot([y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)],
                 [y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)], 'k-')
        plt.ylim(y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min))
        plt.xlim(y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min))
        plt.xlabel('Actual Y')
        plt.ylabel('Predicted Y')
        plt.show()
        
    def sigmaplot(self):
        predicted_y_test, predicted_y_test_std = self.prediction()
        X_train, X_test, y_train, y_test = self.validation()
        
        plt.scatter(predicted_y_test_std, abs(y_test - predicted_y_test), c='blue')
        plt.xlabel('Std. of estimated Y')   
        plt.ylabel('Error of Y')
        plt.show()